- Name: `dissipation-theory--Study-27.ipynb`
- Author: John A. Marohn
- Date: 2024-10-13
- Continued from: ---
- Continued to: ---
- Summary: The pure-Python objects `CantileverModel`, `SampleModel1`, and `SampleModel2` offer a better way to input simulation parameters, because you can input parameters *with units*, but the pure-Python computations are painfully show.   Work out how to pass parameters from `CantileverModel` to `CantileverModelJit`, from `SampleModel1` to `SampleModel1Jit`, and from `SampleModel2` to `SampleModel2Jit`.  We can now enter parameters using the pure-Python objects, then transfer the parameters to the numba/jit objects for fast computations.

In [1]:
from dissipationtheory.constants import ureg
from dissipationtheory.dissipation3 import CantileverModel, SampleModel1, SampleModel2
from dissipationtheory.dissipation3 import CantileverModelJit, SampleModel1Jit, SampleModel2Jit

Use the data in the pure python `CantileverModel` object to initiate a `CantileverModelJit` instance.  Modify the `CantileverModel` code to create a helper function, `.args()`, that spits out a dictionary of parameters that can be fed directly to `CantileverModelJit`.  This new initialization procedure reduces errors that could arise by having to create two cantilever models, one for pure-Python functions and one for numba/jit functions.  The `CantileverModel` is a better way to input cantilever parameters because you input data with units.

In [2]:
cantilever = CantileverModel(
    f_c = ureg.Quantity(75, 'kHz'),
    k_c = ureg.Quantity(2.8, 'N/m'), 
    V_ts = ureg.Quantity(1, 'V'), 
    R = ureg.Quantity(35, 'nm'), 
    d = ureg.Quantity(38, 'nm')
)

In [3]:
cantilever

cantilever

      resonance freq = 75.000 kHz
                     = 4.712e+05 rad/s
     spring constant = 2.800 N/m
  tip-sample voltage = 1.000 V
              radius = 35.000 nm
              height = 38.000 nm

Check that the `.args` function outputs the cantilever parameters as a dictionary, with the parameters converted to SI units and output as plain floats (i.e., no units).

In [4]:
cantilever.args()

{'f_c': 75000.0, 'k_c': 2.8, 'V_ts': 1, 'R': 3.5e-08, 'd': 3.8e-08}

Feed this dictionary to `CantileverModelJit` to initialize the `cantilever_jit` object.

In [5]:
cantilever_jit = CantileverModelJit(**cantilever.args())
cantilever_jit.print()

   cantilever freq =  75000.0 Hz
                   =  471238.89803846896 rad/s
   spring constant =  2.8 N/m
tip-sample voltage =  1.0 V
            radius =  3.5e-08 m
            height =  3.8e-08 m


Now try this same initialization hack with the sample object.  This is trickier because the sample model input has two sets of parameters -- cantilever parameters and sample parameters.  Try to initiate a `SampleModel1Jit` object by passing initialization parameters as a dictionary, really a dictionary of dictionaries, with the cantilever parameters as a dictionary.

In [6]:
cantilever_args = cantilever.args()
sample_args = {
    'cantilever': cantilever_args,
    'h_s': 500e-9,
    'epsilon_s': complex(20, -0.2),
    'sigma': 1e-5,
    'rho': 1e21,
    'epsilon_d': complex(1e6, 0),
    'z_r': 300e-9
    }

In [7]:
sample_jit = SampleModel1Jit(**sample_args)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
non-precise type pyobject
During: typing of argument at <string> (2)

File "<string>", line 2:
<source missing, REPL/exec in use?> 

This error may have been caused by the following argument(s):
- argument 0: Cannot determine Numba type of <class 'dict'>


Failure!  The `SampleModel1Jit` object does not know what to do with the dictionary of cantilever parameters.

Instead of trying to pass the cantilever parameters as a dictionary, try putting the `cantilever_jit` object in this dictionary.
Amazingly, this works!

In [8]:
sample_args = {
    'cantilever': cantilever_jit,
    'h_s': 500e-9,
    'epsilon_s': complex(20, -0.2),
    'sigma': 1e-5,
    'rho': 1e21,
    'epsilon_d': complex(1e6, 0),
    'z_r': 300e-9
    }

In [9]:
sample_jit = SampleModel1Jit(**sample_args)
sample_jit.print()

cantilever
   cantilever freq =  75000.0 Hz
                   =  471238.89803846896 rad/s
   spring constant =  2.8 N/m
tip-sample voltage =  1.0 V
            radius =  3.5e-08 m
            height =  3.8e-08 m

semiconductor
          epsilon (real) =  20.0
          epsilon (imag) =  -0.2
               thickness =  5e-07 m
            conductivity =  1e-05 S/m
          charge density =  1e+21 m^{{-3}}
        reference height =  3e-07 m
 
      roll-off frequency =  1129409.0673730192 Hz
                mobility =  6.241509090043337e-08 m^2/(V s)
      diffusion constant =  1.6135549206567651e-09 m^2/s
            Debye length =  3.7797775481848936e-08 m
        diffusion length =  5.851555252782804e-08 m
effective epsilon (real) =  20.0
effective epsilon (imag) =  -2.5966804779363124

dielectric
 epsilon (real) =  1000000.0
 epsilon (imag) =  0.0
      thickness = infinite


Modify the `cantilever_jit` object and show that the above-generated `sample_jit` does not change.  Conclude that the `sample_jit` is initialized with the values of the `cantilever_jit`-object parameters at initialization time.  They objects are not linked.  Good.

In [10]:
cantilever.k_c = ureg.Quantity(10, 'N/m')
cantilever_jit = CantileverModelJit(**cantilever.args())
cantilever_jit.print()

   cantilever freq =  75000.0 Hz
                   =  471238.89803846896 rad/s
   spring constant =  10.0 N/m
tip-sample voltage =  1.0 V
            radius =  3.5e-08 m
            height =  3.8e-08 m


In [11]:
sample_jit.cantilever.k_c

2.8

Modify the `SampleModel1` code to create a helper function, `.args()`, that spits out a dictionary of parameters that can be fed directly to `SampleModel1Jit`.

In [12]:
sample1 = SampleModel1(
    cantilever = cantilever,
    h_s = ureg.Quantity(500, 'nm'),
    epsilon_s = ureg.Quantity(complex(20, -0.2), ''),
    sigma = ureg.Quantity(1E-5, 'S/m'),
    rho = ureg.Quantity(1e21, '1/m^3'),
    epsilon_d = ureg.Quantity(complex(1e6, 0), ''),
    z_r = ureg.Quantity(300, 'nm')
)

The argument containts an instance of the `CantileverModelJit` for the `cantilever` value.  The other dictionary values are what we expect.

In [13]:
sample1.args()

{'cantilever': <numba.experimental.jitclass.boxing.CantileverModelJit at 0x127470fa0>,
 'h_s': 5.000000000000001e-07,
 'epsilon_s': (20-0.2j),
 'sigma': 1e-05,
 'rho': 1e+21,
 'epsilon_d': (1000000+0j),
 'z_r': 3.0000000000000004e-07}

We can pass this argument list to `SampleModel1Jit` successfully.

In [14]:
sample1_jit = SampleModel1Jit(**sample1.args())
sample1_jit.print()

cantilever
   cantilever freq =  75000.0 Hz
                   =  471238.89803846896 rad/s
   spring constant =  10.0 N/m
tip-sample voltage =  1.0 V
            radius =  3.5e-08 m
            height =  3.8e-08 m

semiconductor
          epsilon (real) =  20.0
          epsilon (imag) =  -0.2
               thickness =  5.000000000000001e-07 m
            conductivity =  1e-05 S/m
          charge density =  1e+21 m^{{-3}}
        reference height =  3.0000000000000004e-07 m
 
      roll-off frequency =  1129409.0673730192 Hz
                mobility =  6.241509090043337e-08 m^2/(V s)
      diffusion constant =  1.6135549206567651e-09 m^2/s
            Debye length =  3.7797775481848936e-08 m
        diffusion length =  5.851555252782804e-08 m
effective epsilon (real) =  20.0
effective epsilon (imag) =  -2.5966804779363124

dielectric
 epsilon (real) =  1000000.0
 epsilon (imag) =  0.0
      thickness = infinite


Try this kind of initialization with `SampleModel2Jit`.

In [15]:
sample2 = SampleModel2(
    cantilever = cantilever,
    epsilon_d = ureg.Quantity(complex(20, -0.2), ''),
    h_d = ureg.Quantity(1, 'nm'),
    epsilon_s = ureg.Quantity(complex(20, -0.2), ''),
    sigma = ureg.Quantity(1E-5, 'S/m'),
    rho = ureg.Quantity(1e21, '1/m^3'),
    z_r = ureg.Quantity(300, 'nm')
)

In [16]:
sample2_jit = SampleModel2Jit(**sample2.args())
sample2_jit.print()

cantilever
   cantilever freq =  75000.0 Hz
                   =  471238.89803846896 rad/s
   spring constant =  10.0 N/m
tip-sample voltage =  1.0 V
            radius =  3.5e-08 m
            height =  3.8e-08 m

dielectric
 epsilon (real) =  20.0
 epsilon (imag) =  -0.2
      thickness =  1e-09 m

semiconductor
          epsilon (real) =  20.0
          epsilon (imag) =  -0.2
               thickness = infinite
            conductivity =  1e-05 S/m
                mobility =  6.241509090043337e-08 m^2/(V s)
        reference height =  3.0000000000000004e-07 m
 
      roll-off frequency =  1129409.0673730192 Hz
      diffusion constant =  1.6135549206567651e-09 m^2/s
          charge density =  1e+21 m^{{-3}}
            Debye length =  3.7797775481848936e-08 m
        diffusion length =  5.851555252782804e-08 m
effective epsilon (real) =  20.0
effective epsilon (imag) =  -2.5966804779363124
